In [1]:
import requests
import pandas as pd

import numpy as np
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import sqlite3 as lite
import time
from dateutil.parser import parse
import collections

In [2]:
r = requests.get('https://www.divvybikes.com/stations/json')

In [3]:
r.headers

{'X-Powered-By': 'PHP/5.4.45', 'Transfer-Encoding': 'chunked', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding,User-Agent', 'Connection': 'close', 'Server': 'Apache/2.2.15 (Red Hat)', 'Last-Modified': 'Thu, 09 Jun 2016 21:41:02 GMT', 'x-version': '1.0', 'Date': 'Thu, 09 Jun 2016 21:41:39 GMT', 'Content-Type': 'application/json'}

In [4]:
key_list = [] #unique list of keys for each station listing
for station in r.json()['stationBeanList']:
    for k in station.keys():
        if k not in key_list:
            key_list.append(k)

In [5]:
r.json()['executionTime']

u'2016-06-09 04:41:02 PM'

In [6]:
r.json()['stationBeanList'][0]

{u'altitude': u'',
 u'availableBikes': 5,
 u'availableDocks': 28,
 u'city': u'Chicago',
 u'id': 2,
 u'is_renting': True,
 u'landMark': u'541',
 u'lastCommunicationTime': u'06/09/2016 04:40:20 PM',
 u'latitude': 41.876428,
 u'location': u'',
 u'longitude': -87.620339,
 u'postalCode': u'',
 u'stAddress1': u'Buckingham Fountain',
 u'stAddress2': u'',
 u'stationName': u'Buckingham Fountain',
 u'status': u'IN_SERVICE',
 u'statusKey': 1,
 u'statusValue': u'In Service',
 u'testStation': False,
 u'totalDocks': 35}

In [7]:
r.json().keys()

[u'executionTime', u'stationBeanList']

In [8]:
unique=[]
A = [(station.keys(),station.values()) for station in r.json()['stationBeanList']]

In [9]:
df = json_normalize(r.json()['stationBeanList'])
df.head(10)

,altitude,availableBikes,availableDocks,city,id,is_renting,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,status,statusKey,statusValue,testStation,totalDocks
0,,5,28,Chicago,2,True,541,06/09/2016 04:40:20 PM,41.876428,,-87.620339,,Buckingham Fountain,,Buckingham Fountain,IN_SERVICE,1,In Service,False,35
1,,0,31,Chicago,3,True,544,06/09/2016 04:37:01 PM,41.867226,,-87.615355,,Shedd Aquarium,,Shedd Aquarium,IN_SERVICE,1,In Service,False,31
2,,13,9,Chicago,4,True,545,06/09/2016 04:36:54 PM,41.856268,,-87.613348,,Burnham Harbor,,Burnham Harbor,IN_SERVICE,1,In Service,False,23
3,,5,17,,5,True,030,06/09/2016 04:38:16 PM,41.874053,620 S. State St.,-87.627716,,State St & Harrison St,,State St & Harrison St,IN_SERVICE,1,In Service,False,23
4,,2,28,Chicago,6,True,548,06/09/2016 04:37:32 PM,41.885042,,-87.612795,,Dusable Harbor,,Dusable Harbor,IN_SERVICE,1,In Service,False,31
5,,15,3,Chicago,7,True,534,06/09/2016 04:39:37 PM,41.886349,,-87.617517,,Field Blvd & South Water St,,Field Blvd & South Water St,IN_SERVICE,1,In Service,False,19
6,,8,11,Chicago,9,True,695,06/09/2016 04:40:26 PM,41.828792,NA1,-87.680604,,Leavitt St & Archer Ave,,Leavitt St & Archer Ave,IN_SERVICE,1,In Service,False,19
7,,6,4,Chicago,11,True,496,06/09/2016 04:38:43 PM,41.766638,,-87.576450,,Jeffery Blvd & 71st St,,Jeffery Blvd & 71st St,IN_SERVICE,1,In Service,False,11
8,,6,8,Chicago,12,True,497,06/09/2016 04:36:21 PM,41.766409,MLSW,-87.565688,,South Shore Dr & 71st St,,South Shore Dr & 71st St,IN_SERVICE,1,In Service,False,15
9,,14,11,Chicago,13,True,066,06/09/2016 04:37:31 PM,41.932418,2790 N.Wilton Ave,-87.652705,,Wilton Ave & Diversey Pkwy,,Wilton Ave & Diversey Pkwy,IN_SERVICE,1,In Service,False,27


In [10]:
A = r.json()['stationBeanList'][0]
A['id'],A['totalDocks']
tuple([A[ref_vars[x]] for x in range(len(ref_vars))])

NameError: name 'ref_vars' is not defined

In [ ]:
len(ref_vars)

In [ ]:
df['availableBikes'].hist()
plt.show()

In [16]:
df['totalDocks'].hist()
plt.show()

In [33]:
df[df['statusKey']>1]

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks


In [58]:
df['totalDocks'].mean() , df['totalDocks'].median()

(17.52, 15.0)

In [40]:
df['availableBikes'].mean()

6.671578947368421

In [41]:
df['availableBikes'].median()

6.0

In [19]:
#Median number of available bikes in In service stations
df[df['statusValue']=='In Service']['availableBikes'].median()

7.0

In [74]:
df1 =df
df2 = df['availableBikes']

In [4]:
sql = '''CREATE TABLE citibike_reference (
    id INT PRIMARY KEY,
    totalDocks INT,
    city TEXT,
    altitude INT, 
    stAddress2 TEXT,
    longitude NUMERIC,
    postalCode TEXT,
    testStation TEXT,
    stAddress1 TEXT,
    stationName TEXT,
    landMark TEXT,
    latitude NUMERIC,
    location TEXT
)'''

In [3]:
con = lite.connect('divvy_bike.db')
cur = con.cursor()

In [11]:
with con:
    cur.execute(sql)

In [19]:
insert_sql = '''INSERT INTO citibike_reference 
(id, totalDocks, city, altitude, stAddress2, longitude, 
postalCode, testStation, stAddress1, stationName, landMark, 
latitude, location) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'''

ref_variables = tuple(['id', 'totalDocks', 'city', 'altitude', 'stAddress2', 
              'longitude', 'postalCode', 'testStation', 'stAddress1', 
              'stationName', 'landMark', 'latitude', 'location'])
I=[]
for i in ref_variables:
    I.append(unicode(i))

In [20]:
#[station[I[11]] for station in rjson['stationBeanList']]


In [21]:
#Input list of unique station keys for each station. 
#This list is the same for all stations
A = [np.int(station['id']) for station in r.json()['stationBeanList']]
B = [station for station in r.json()['stationBeanList']]

In [22]:
with con:
    for station in r.json()['stationBeanList']:
        cur.execute(sql,(station['id'],station['totalDocks'],
                    station['city'],station['altitude'],
                    station['stAddress2'],
                    station['longitude'],station['postalCode'],
                    station['testStation'],station['stAddress1'],
                    station['stationName'],station['landMark'],
                    station['latitude'],station['location']))

IntegrityError: UNIQUE constraint failed: citibike_reference.id

In [4]:
station_ids = df['id'].tolist()

In [5]:
station_ids = ['_' + str(x) + ' INT' for x in station_ids]

In [2]:
#cur.execute string, attaching the station_ids list 
S = "create table available_bikes(execution_time INT, "+", ".join(station_ids)+")"

NameError: name 'station_ids' is not defined

In [18]:
with con:
    cur.execute(S)

In [19]:
exec_time = parse(r.json()['executionTime'])
exec_time.strftime('%s')

'1441212181'

In [20]:
pd_exec_time=pd.to_datetime(r.json()['executionTime'])
#pd_exec_time.strftime('%Y-%m-%dT%H:%M:%S')
pd_exec_time.strftime('%s')

'1441212181'

In [21]:
r.json()['executionTime']

u'2015-09-02 11:43:01 AM'

Due to differences in operating systems, 
you may need to check out the documentation for strftime() 
if you run into errors. Other students have reported that 
(exec_time - datetime.datetime(1970,1,1)).total_seconds() or 
replacing %s with %Y-%m-%dT%H:%M:%S works on Windows. 
As always, refer to the Thinkful Students 
Community to work with others to get help and squash bugs.

In [22]:
sql = '''INSERT INTO available_bikes 
(execution_time) VALUES (?)', (exec_time.strftime('%s'),)'''

In [23]:
with con:
    cur.execute('insert into available_bikes (execution_time) values (?)',(exec_time.strftime('%s'),))

In [55]:
df = pd.read_json(r.text)
pd.Dataframe(df['stationBeanList'],orient='split')

AttributeError: 'module' object has no attribute 'Dataframe'

In [7]:
#%%file citybike.py
#!usr/bin/env python
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import sqlite3 as lite
import time
from dateutil.parser import parse
import collections


def load_dataset(loc):
    r = requests.get(loc)
    return r.json()

def preprocess(rjson,**kwargs):
    keys = rjson.keys()
    df = json_normalize(rjson[kwargs['column']])
    return keys , df

def drop_table(name):
    return 'DROP TABLE IF EXISTS'+' '+name
    
def create_table(con,**kwargs):
    cur = con.cursor()
    with con:
        cur.execute(kwargs["drop_table"])
        cur.execute(kwargs["sql_string"])
    print('Database created')

def ref_data(con,rjson,**kwargs):
    cur = con.cursor()
    var = kwargs['ref_vars']
    insert_sql='INSERT INTO '+kwargs['table']+' ('+', '.join(var)+') VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'
    with con:
        for station in rjson[kwargs['column']]:
            cur.execute(insert_sql,tuple([station[var[i]] for i in range(len(var))]))

            
def time_data(con,rjson,**kwargs):
    cur = con.cursor()
    exec_time=pd.to_datetime(rjson[kwargs['column']])
    insert_sql='insert into '+kwargs['table']+' ('+kwargs['column']+') values (?)'
    id_bikes = collections.defaultdict(int)
    for station in rjson['stationBeanList']:
        id_bikes[station['id']]=station['availableBikes']
    with con:
        cur.execute(insert_sql,(exec_time.strftime('%s'),))
        for k,v in id_bikes.iteritems():
            cur.execute('update available_bikes set _'+str(k)+' = '+str(v)+' where executionTime = '
                        + exec_time.strftime('%s')+ ';')
    
if __name__ =='__main__':
    
    loc = 'https://www.divvybikes.com/stations/json'
    
    #JSON output using requests
    rjson = load_dataset(loc)
    keys, df = preprocess(rjson,column='stationBeanList')
    
    ref_table_sql='''CREATE TABLE citibike_reference (
    id INT PRIMARY KEY,
    totalDocks INT,
    city TEXT,
    altitude INT, 
    stAddress2 TEXT,
    longitude NUMERIC,
    postalCode TEXT,
    testStation TEXT,
    stAddress1 TEXT,
    stationName TEXT,
    landMark TEXT,
    latitude NUMERIC,
    location TEXT
    )'''
    
    ref_vars = '''id, totalDocks, city, altitude, stAddress2, longitude, 
    postalCode, testStation, stAddress1, stationName, landMark, 
    latitude, location'''.replace('\n','').replace('    ','').replace(' ','').split(',')
                        
    #insert_sql = 'INSERT INTO citibike_reference'+' ('+', '.join(ref_vars)+') VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'              
    
    db_name='divvy_bike.db'
    
    #Create a database
    con = lite.connect(db_name)
    
    #Reference Table
    create_table(con,sql_string=ref_table_sql,drop_table=drop_table('citibike_reference'))
    ref_data(con,rjson,table='citibike_reference',ref_vars=ref_vars,column='stationBeanList')
    
    #Time Table
    station_ids = df['id'].tolist()
    station_ids = ['_' + str(x) + ' INT' for x in station_ids]
    
    time_table_sql="create table available_bikes (executionTime INT, "+", ".join(station_ids)+");"
    
    create_table(con,sql_string=time_table_sql,drop_table=drop_table('available_bikes'))
    
    time_data(con,rjson,table='available_bikes',column='executionTime')
    #create_table(con,sql_string=time_table,drop_table=drop_table('available_bikes'))
    df1 = pd.read_sql('SELECT * from available_bikes',con)

Overwriting citybike.py


In [24]:
ref_vars = '''id, totalDocks, city, altitude, stAddress2, longitude, 
    postalCode, testStation, stAddress1, stationName, landMark, 
    latitude, location'''.replace('\n','').replace('    ','').replace(' ','').split(',')


In [30]:
'''id, totalDocks, city, altitude, stAddress2, longitude, 
    postalCode, testStation, stAddress1, stationName, landMark, 
    latitude, location'''.replace('\n','').strip().split(',')

['id',
 ' totalDocks',
 ' city',
 ' altitude',
 ' stAddress2',
 ' longitude',
 '     postalCode',
 ' testStation',
 ' stAddress1',
 ' stationName',
 ' landMark',
 '     latitude',
 ' location']

In [3]:
df1 = pd.read_sql('SELECT * from available_bikes',con)

In [47]:
t = pd.to_datetime(rjson['executionTime'])
cur.execute?

In [36]:
[(i['id'],i['availableBikes']) for i in rjson['stationBeanList']][:5]

[(2, 6), (3, 9), (4, 7), (5, 15), (6, 7)]

In [6]:
df1

,executionTime,_2,_3,_4,_5,_6,_7,_9,_11,_12,...,_502,_503,_504,_505,_506,_507,_508,_509,_510,_511
0,1445252882,6,14,13,5,9,14,12,6,10,...,15,10,7,9,3,4,8,5,9,7


In [66]:
df.groupby('id').median().head(10)

,availableBikes,availableDocks,latitude,longitude,statusKey,testStation,totalDocks
id,,,,,,,
2,3,32,41.876428,-87.620339,1,False,35
3,0,31,41.867226,-87.615355,1,False,31
4,1,22,41.856268,-87.613348,1,False,23
5,2,21,41.874053,-87.627716,1,False,23
6,0,29,41.885042,-87.612795,1,False,31
7,6,13,41.886349,-87.617517,1,False,19
9,8,10,41.828792,-87.680604,1,False,19
11,5,6,41.766638,-87.576450,1,False,11
12,8,7,41.766409,-87.565688,1,False,15


In [17]:
from datetime import datetime
datetime.strptime?

In [71]:
df1 = df.groupby('id').median()
#df1.drop(df1.columns[[1,2,3,4,5,6]],1)

In [56]:
S = "UPDATE available_bikes SET _" + str(k) + " = " + str(v) + " WHERE execution_time = " + exec_time.strftime('%s') + ";"

NameError: name 'k' is not defined

In [65]:
id_bikes = collections.defaultdict(int) #defaultdict to store available bikes by station
exec_time=pd.to_datetime(rjson['executionTime'])
#loop through the stations in the station list
for station in rjson['stationBeanList']:
    id_bikes[station['id']] = station['availableBikes']

#iterate through the defaultdict to update the values in the database
with con:
    for k, v in id_bikes.iteritems():
        cur.execute("UPDATE available_bikes SET _" + str(k) + " = " + str(v) + " WHERE executionTime = " + exec_time.strftime('%s') + ";")
        

In [36]:
def insert_data(con,cur,rjson,**kwargs):
    var = kwargs['ref_vars']
    insert_sql='INSERT INTO '+kwargs['table']+' ('+', '.join(var)+') VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'
    with con:
        for station in rjson[kwargs['column']]:
            cur.execute(insert_sql,tuple([station[var[i]] for i in range(len(var))]))
    
insert_data(con,cur,rjson,table='available_bikes',column='executionTime')

KeyError: 'ref_vars'

In [10]:
rjson['executionTime']

u'2015-09-02 09:37:31 PM'

In [7]:
station_id_col = ["_"+str(x)+' INT' for x in df['id'].tolist()]
#station_id_col


In [39]:
insert_sql = 'INSERT INTO citibike_reference'+' ('+', '.join(ref_vars)+') VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'
insert_sql

'INSERT INTO citibike_reference (id, totalDocks, city, altitude, stAddress2, longitude, postalCode, testStation, stAddress1, stationName, landMark, latitude, location) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'

In [43]:
ref_vars = '''id, totalDocks, city, altitude, stAddress2, longitude, 
    postalCode, testStation, stAddress1, stationName, landMark, 
    latitude, location'''.replace('\n','').replace('    ','').replace(' ','').split(',')
ref_vars

['id',
 'totalDocks',
 'city',
 'altitude',
 'stAddress2',
 'longitude',
 'postalCode',
 'testStation',
 'stAddress1',
 'stationName',
 'landMark',
 'latitude',
 'location']

In [32]:
A = dict(ref_vars=ref_vars)
A['ref_vars']

['id',
 'totalDocks',
 'city',
 'altitude',
 'stAddress2',
 'longitude',
 'postalCode',
 'testStation',
 'stAddress1',
 'stationName',
 'landMark',
 'latitude',
 'location']

In [5]:
def drop_table(name):
    return 'DROP TABLE IF EXISTS'+' '+name

In [5]:
from odo import odo
import blaze as blz

In [8]:
data = blz.Data('/home/gpk/thinkful/divvy_bike.db')

NotImplementedError: Unable to parse uri to data resource: /home/gpk/thinkful/divvy_bike.db

In [8]:
A  = dict(column='stationBeanList')

In [101]:
def insert_data(con,cur,df,**kwargs):
    for i,item in enumerate(kwargs['ref_vars']):
        with con:
            cur.execute(kwargs['insert_sql'],])

In [98]:
ref_vars = '''id, totalDocks, city, altitude, stAddress2, longitude, 
    postalCode, testStation, stAddress1, stationName, landMark, 
    latitude, location'''.replace('\n','').split(',')
ref_vars[12].strip()

'location'

In [104]:
S = "create table available_bikes(execution_time INT, "+", ".join(station_id_col)+")"

len(ref_vars)

13

In [15]:
np.math.factorial